In [ ]:
import os
import numpy as np
from PIL import Image

def ProjectDataLoader(folder_path):
    images = []
    labels = []

    for fname in os.listdir(folder_path):
        if fname.endswith(".png"):
            # Format: digit-groupID-memberID.png
            digit = int(fname.split("-")[0])
            labels.append(digit)

            img_path = os.path.join(folder_path, fname)
            img = Image.open(img_path).convert("L")  # grayscale
            img = img.resize((28, 28))               # ensure MNIST size
            img_np = np.array(img, dtype=np.float32)
            images.append(img_np)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [ ]:
from torchvision import datasets, transforms
import torch

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
from torch import nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return F.log_softmax(self.out(x), dim=1)

In [ ]:
model = MLP()
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 20

for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(trainloader):.3f}")

In [ ]:
testset = datasets.MNIST(
    '~/.pytorch/MNIST_data/',
    train=False,
    download=True,
    transform=transform
)
testloader = torch.utils.data.DataLoader(testset, batch_size=64)

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in testloader:
        logps = model(images)
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)

        correct += (top_class.squeeze() == labels).sum().item()
        total += labels.size(0)

print("MNIST Test Accuracy:", correct / total)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

imgs, lbls = ProjectDataLoader("C:/Users/923dy/CSCI580_Fall25_Group4/digits")

imgs_tensor = torch.stack([transform(img) for img in imgs])

correct = 0
model.eval()

with torch.no_grad():
    logps = model(imgs_tensor)
    ps = torch.exp(logps)
    preds = ps.argmax(dim=1)

    correct = (preds == torch.tensor(lbls)).sum().item()

print(f"Team Images Accuracy: {correct / len(lbls)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def plot_prediction(image_tensor, probs):
    """
    image_tensor : shape (1, 28, 28)
    probs        : softmax probabilities shape (10,)
    """

    image_np = image_tensor.squeeze().cpu().numpy()

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    # --- LEFT: the digit image ---
    ax1.imshow(image_np, cmap="viridis")
    ax1.axis("off")

    # --- RIGHT: Probability bars ---
    y_pos = np.arange(10)
    ax2.barh(y_pos, probs)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([str(d) for d in range(10)])
    ax2.set_xlim(0, 1)
    ax2.set_title("Class Probability")

    plt.tight_layout()
    plt.show()

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

imgs, lbls = ProjectDataLoader("C:/Users/923dy/CSCI580_Fall25_Group4/digits")

imgs_tensor = torch.stack([transform(img) for img in imgs])

correct = 0
model.eval()

with torch.no_grad():
    logps = model(imgs_tensor)
    ps = torch.exp(logps)
    preds = ps.argmax(dim=1)

    correct = (preds == torch.tensor(lbls)).sum().item()

print(f"Team Images Accuracy: {correct / len(lbls)}")

# ---------- Plot results for each image ----------
for i in range(len(imgs_tensor)):
    img_t = imgs_tensor[i]
    logp  = logps[i]
    p     = torch.exp(logp).cpu().numpy()

    print(f"Image {i} --- True: {lbls[i]}, Predicted: {np.argmax(p)}")
    plot_prediction(img_t, p)